In [1]:
import torch
import numpy as np
import pandas as pd
import re
import torch
import sklearn
import nltk
import joblib
import pandas as pd

from transformers import GPT2Tokenizer, T5ForConditionalGeneration 
from transformers import AutoTokenizer, T5ForConditionalGeneration
from transformers import set_seed

import clickhouse_connect

from sentence_transformers import SentenceTransformer
from FlagEmbedding import BGEM3FlagModel

from sklearn.neighbors import NearestNeighbors

from utils.utils import Chunker, Generator, Retriever
from utils.llm_config import GenerationConfig

device = 'cuda'

set_seed(42)

nltk.download('punkt')
torch.cuda.is_available()

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2024-03-13 18:22:02.627578: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
client = clickhouse_connect.get_client(host='62.84.115.43', port=8123, username='default', password='g5oHPYX4', database='dev')

In [3]:
bge_retriever_model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True, pooling_method="cls")

Fetching 23 files:   0%|          | 0/23 [00:00<?, ?it/s]

imgs/.DS_Store:   0%|          | 0.00/6.15k [00:00<?, ?B/s]

imgs/bm25.jpg:   0%|          | 0.00/69.0k [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

sparse_linear.pt:   0%|          | 0.00/3.52k [00:00<?, ?B/s]

loading existing colbert_linear and sparse_linear---------


In [ ]:
e5_retriever_model = SentenceTransformer("intfloat/multilingual-e5-large", device='cuda')
e5_retriever_model.max_seq_length = 512


generator_model_name = 'hivaze/AAQG-QA-QG-FRED-T5-1.7B'
generator_tokenizer = AutoTokenizer.from_pretrained(generator_model_name)
generator_model = T5ForConditionalGeneration.from_pretrained(generator_model_name).cuda().eval()

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
generator_config = GenerationConfig()
generator = Generator(generator_tokenizer, generator_model, config=generator_config)

In [ ]:
queries = pd.read_csv('Book1 (1).csv', sep=';')
queries.site_id = queries.site_id.apply(lambda x: x if  x[-1] == '/' else x+'/')
queries.site_id = queries.site_id.apply(lambda x: x if 'http://pravo.gov.ru' in x else 'https://cbr.ru' + x)

## Chunker

In [ ]:
chunker = Chunker(max_chunk_len=2500, overlap_len=500)


texts = client.query_df('select * from document').set_index('url')
chunks, urls = chunker.split_texts(texts)
client.insert(table='chunk', data=[chunks, urls], column_names=['text', 'url'], column_oriented=True)

In [ ]:
# encoded_queries = bge_retriever_model.encode(book["query"].tolist(), return_dense=True, return_sparse=False, return_colbert_vecs=True, batch_size=16, max_length=512)

In [ ]:
chunks_df = client.query_df('select * from chunk').set_index('uuid')

bge_embeddings = bge_retriever_model.encode(chunks_df.text.tolist(), return_dense=True, return_sparse=False, return_colbert_vecs=True, batch_size=64, max_length=512)
e5_embeddings = e5_retriever_model.encode(chunks_df.text.tolist(), batch_size=16, normalize_embeddings=True, show_progress_bar=True)

client.insert(table='chunk_embedding',
              data=[chunks_df.index.tolist(), 
                    bge_embeddings['colbert_vecs'].tolist(), 
                    e5_embeddings.tolist(),
                    bge_embeddings['dense_vecs'].to_list()],
              column_names=['chunk_uuid', 
                            'bge_m3_embeddings', 
                            'e5_embeddings', 
                            'bge_m3'], column_oriented=True)

In [35]:
client.query_df('select * from chunk_embedding')

,chunk_uuid,bge_m3_embedding,e5_embedding,bge_m3
0,7a0af8f4-95ed-48e5-916b-232f95e1e38f,"[0.007940628, 0.024005534, -0.004237408, 0.025...","[0.026318824, 0.008612728, -0.0062181787, -0.0...",[]
1,bc9feeec-1e5c-4284-916c-51d52e81ed04,"[-0.05350949, 0.021115648, -0.048056465, 0.011...","[-0.014645152, -0.02507355, -0.034141522, -0.0...",[]
2,c9035816-39b6-4c35-916c-abbf2eea058a,"[-0.035761513, 0.008931544, 0.029431168, -0.01...","[0.020523636, 0.009849383, 0.0036926188, -0.05...",[]
3,175899d5-52fd-4c25-916c-de6bc9ae03e4,"[-0.028674705, 0.00039257374, -0.03999512, -0....","[0.0017483772, -0.040360793, -0.03389109, -0.0...",[]
4,f2c26c9e-bb57-4a81-916d-46b6b9e49d43,"[-0.018863013, -0.00924226, -0.03296817, 0.035...","[0.002780524, -0.01835111, -0.030327525, -0.04...",[]
...,...,...,...,...
1306,e7bc08f2-e1da-46f4-916a-9b34fe10ef18,"[-0.018292995, 0.0015480752, -0.01335002, 0.03...","[0.01271502, -0.016061038, -0.016362324, -0.02...",[]
1307,123fd6aa-5012-4556-916a-b2e72e5be455,"[-0.0010022236, -0.0011319566, -0.008193335, -...","[0.036662173, -0.019630207, -0.030757874, -0.0...",[]
1308,b88beb22-9754-4ace-916a-b728788d3fda,"[-0.05853829, -0.0031400016, -0.0030861655, -6...","[-0.007210687, -0.0043519395, -0.027687693, -0...",[]
1309,50ab90db-f370-4002-916a-e430c840e2b9,"[-0.023804326, -0.013488999, 0.021434963, -0.0...","[-0.0012883231, -0.012499608, 0.0007025175, -0...",[]


In [30]:
client.insert(table='chunk_embedding',
              data=[chunks_df.index.tolist(), 
                    bge_embeddings.tolist(), 
                    e5_embeddings.tolist()],
              column_names=['chunk_uuid', 
                            'bge_m3_embedding', 
                            'e5_embedding'], column_oriented=True)

## Retriever & Generator

In [7]:
retriever_model = {'bge_m3': bge_retriever_model, 'e5': e5_retriever_model}

In [8]:
retriever = Retriever(retriever_model, client)

In [16]:
question = queries.question.sample(1).item()

In [17]:
question

'Какие условия совершения операций по выдаче микрозаймов микрофинансовой организацией?'

In [26]:
retriever.get_neighbors(question)

(['(В редакции Федерального закона от 23.07.2013 No 251-ФЗ)2.\nМикрофинансовая организация обязана:1) предоставить лицу, подавшему заявление на предоставление микрозайма, полную и достоверную информацию о порядке и об условиях предоставления микрозайма, о его правах и обязанностях, связанных с получением микрозайма; (В редакции Федерального закона от 29.12.2015 No 407-ФЗ)2) разместить копию правил предоставления микрозаймов в месте, доступном для обозрения и ознакомления с ними любого заинтересованного лица, и в сети Интернет;21) разместить сведения о полученном предписании Банка России, ограничивающем деятельность микрофинансовой организации, предусмотренном пунктом 71 части 4 статьи 14 настоящего Федерального закона, в местах обслуживания клиентов и на официальном сайте этой микрофинансовой организации в информационно-телекоммуникационной сети "Интернет" (при наличии) не позднее дня, следующего за днем получения такого предписания; (Дополнение пунктом - Федеральный закон от 13.07.202

In [24]:
print(generator.get_answer(question, *retriever.get_neighbors(question, k=3), temperature=0.8))

Микрофинансовая организация вправе запрашивать у лица, подавшего заявление на предоставление микрозайма, документы и сведения

Использованные документы:
1) http://pravo.gov.ru/proxy/ips/?docbody=&link_id=0&nd=102139631&bpa=cd00000&bpas=cd00000& 

